In [3]:
import sys
import pandas as pd
import dateutil
import numpy as np
sys.path.append('../')
from utilities.era5_down import era5_down
import glob

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
cs_file = '../data/loc_stakes_SMB.csv'
dfmb = pd.read_csv(cs_file,
   delimiter='\t', index_col=['Date/Time'],
    parse_dates=['Date/Time'], na_values='NAN',date_parser=date_parser)

In [5]:
labels = dfmb.drop_duplicates(subset=['Event'])

In [6]:
t2m = 't2m'
rh2 = 'rh'
ff  = 'u2'
Prec = 'tp'
Snowfall = 'sf'
SWD = 'SWin'
LWD = 'LWin'
Pres = 'press'
fcc  = 'tcc'
msl  = 'msl'

In [5]:
files = sorted(glob.glob('../data/ERA_59_20_day/*.nc'))
len(files)

62

In [8]:
stake_inp = []
for i in labels['Event']:#[0:2]    
    stake = dfmb.loc[lambda df: df['Event'] == i, :]
    df_day = era5_down(files, stake['Longitude'][0], 
               stake['Latitude'][0], stake['Elevation [m]'][0])
    df_day['t2m_an'] = (df_day[t2m] - df_day[t2m].mean())/df_day[t2m].std()
    str_date = stake.index.values[0]
    end_date = stake.index.values[1]
    subset = df_day.loc[str_date:end_date].copy()
    subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0).where(subset[t2m] <= 0, 1)
    subset['stake'] = stake['Event'][0]
    subset_f = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                     SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                     Snowfall:nansumwrapper, msl:np.mean,
                                     Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                     'PDD':nansumwrapper})
    subset_f.index = ([str_date])
    subset_f0 = []
    for i in range(len(stake)-1):
        str_date = stake.index.values[i]
        end_date = stake.index.values[i+1]
        subset = df_day.loc[str_date:end_date].copy()
        subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0).where(subset[t2m] <= 0, 1)
        subset['stake'] = stake['Event'][0]
        subset0 = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                         SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                         Snowfall:nansumwrapper, msl:np.mean,
                                         Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                         'PDD':nansumwrapper})
        subset0.index = ([end_date])
        subset_f0.append(subset0)
    subset_f0 = pd.concat(subset_f0)
    subset_f[[Prec, 'PDD']] = 0.0
    subset_ff = pd.concat([subset_f, subset_f0])
    subset_fff = pd.concat([stake, subset_ff], axis=1)
    stake_inp.append(subset_fff)


In [9]:
stake_inp = pd.concat(stake_inp)
stake_inp.index.name = 'Date/Time'

In [10]:
stake_inp.to_csv('../data/SMB_input_2011_2016_ERA5.csv', index=True, sep='\t')